## UnSu Unet Object Localization

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image
from tensorflow.keras.initializers import he_uniform
from sklearn.model_selection import train_test_split


import numpy as np
import matplotlib.pyplot as plt

2024-10-17 22:30:14.377458: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 22:30:14.407871: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-17 22:30:14.408279: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-17 22:30:15.039020: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Import custom functions to extract our Image arrays and Pixel Mask arrays from our created fits files dataset
from dataGathering import extractImageArray, extractPixelMaskArray, extract_star_catalog, getStarData

# Import astropy to read fits files, and os to interact with the file system
from astropy.io import fits
import os

In [3]:
getStarData('II/246', 10, 'data')

SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  961
Number of cataloged stars in image:  596
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  844
Number of cataloged stars in image:  552
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  687
Number of cataloged stars in image:  468
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  1884
Number of cataloged stars in image:  1273
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  2207
Number of cataloged stars in image:  1416
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  735
Number of cataloged stars in image:  474
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  457
Number of cataloged stars in image:  279
Drawing
SkyView
Vizier
Save
Save Catalog
Number of stars in catalog query:  1006
Number of cataloged stars in image:  670
Drawing
SkyView
Vizier
Save
Save Ca

In [4]:
# Create images and masks arrays lists
images = []
masks = []

# Create df to store the star data inside each fits file
star_data = []

# Create a list of all the fits files in the dataset folder
fits_files = os.listdir('data/')

# For all the fits files in the dataset folder specified in file_path, extract the image and mask arrays to the respective lists
file_path = 'data/'
for file in os.listdir(file_path):
    if file.endswith('.fits'):
        images.append(extractImageArray(file_path + file))
        masks.append(extractPixelMaskArray(file_path + file))
        star_data.append(extract_star_catalog(file_path + file))

        print(file + ' added to dataset')

data7.fits added to dataset
data4.fits added to dataset
data5.fits added to dataset
data8.fits added to dataset
data1.fits added to dataset
data9.fits added to dataset
data6.fits added to dataset
data3.fits added to dataset
data0.fits added to dataset
data2.fits added to dataset


In [5]:
masks[0].shape

(512, 512)

In [6]:
# Encoder path
    #32 kernels
    #3x3 kernel size
    #padding = same considers edges in the input
def unet_model(input_shape):
    inputs = layers.Input(shape=input_shape)

# Encoder path
    c1 = layers.Conv2D(filters[0], kernel_size, activation=activation, padding=padding, kernel_initializer=initializer)(inputs)
    c1 = layers.Conv2D(filters[0], kernel_size, activation=activation, padding=padding, kernel_initializer=initializer)(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(p4)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c5)

    # Decoder path
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(u9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer=he_uniform())(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9) # 512 x 512 x 1 filled with 1s or 0s

    model = models.Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [7]:
# Convert lists to numpy arrays
# This ensures that the data is in the correct format for train_test_split and model.fit.
train_images = np.array(images)
train_masks = np.array(masks)

In [8]:
# Ensure the input data has the correct shape
if train_images.ndim == 3:  # If images are grayscale and have shape (num_samples, height, width)
    train_images = np.expand_dims(train_images, axis=-1)  # Add channel dimension
if train_masks.ndim == 3:  # If masks are grayscale and have shape (num_samples, height, width)
    train_masks = np.expand_dims(train_masks, axis=-1)  # Add channel dimension

In [9]:
# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(train_images, train_masks, test_size=0.2, random_state=0)

In [10]:
# Duplicate channels for the grayscale images since the model expects 3 channels
train_images = np.repeat(train_images, 3, axis=-1)
val_images = np.repeat(val_images, 3, axis=-1)

In [11]:
# Show distribution of training and validation sets
print('Training images: ', train_images.shape)
print('Training masks: ', train_masks.shape)
print('Validation images: ', val_images.shape)
print('Validation masks: ', val_masks.shape)

Training images:  (8, 512, 512, 3)
Training masks:  (8, 512, 512, 1)
Validation images:  (2, 512, 512, 3)
Validation masks:  (2, 512, 512, 1)


In [12]:
# Create and compile the model
input_shape = (512, 512, 3)
model = unet_model(input_shape)
model.compile(
optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

2024-10-17 22:31:24.555374: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-17 22:31:24.555805: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
history = model.fit(train_images, train_masks, 
                    validation_data=(val_images, val_masks), 
                    epochs=100, 
                    batch_size=32)

Epoch 1/100


2024-10-17 22:31:37.362875: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 536870912 exceeds 10% of free system memory.
2024-10-17 22:31:39.860289: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 603979776 exceeds 10% of free system memory.
2024-10-17 22:31:39.864850: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 536870912 exceeds 10% of free system memory.
2024-10-17 22:31:39.864922: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 603979776 exceeds 10% of free system memory.


1/1 [==============================] - 21s 21s/step - loss: 1652.7817 - accuracy: 0.2425 - val_loss: 31.7011 - val_accuracy: 0.9674
Epoch 2/100


2024-10-17 22:31:55.926010: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 536870912 exceeds 10% of free system memory.


1/1 [==============================] - 17s 17s/step - loss: 58.9418 - accuracy: 0.9642 - val_loss: 15.4374 - val_accuracy: 0.9978
Epoch 3/100
1/1 [==============================] - 16s 16s/step - loss: 37.2845 - accuracy: 0.9966 - val_loss: 18.3671 - val_accuracy: 0.9980
Epoch 4/100
1/1 [==============================] - 14s 14s/step - loss: 43.9654 - accuracy: 0.9967 - val_loss: 19.7028 - val_accuracy: 0.9980
Epoch 5/100
1/1 [==============================] - 14s 14s/step - loss: 46.9025 - accuracy: 0.9967 - val_loss: 19.7645 - val_accuracy: 0.9980
Epoch 6/100
1/1 [==============================] - 14s 14s/step - loss: 46.9793 - accuracy: 0.9967 - val_loss: 18.5095 - val_accuracy: 0.9980
Epoch 7/100
1/1 [==============================] - 14s 14s/step - loss: 43.9946 - accuracy: 0.9967 - val_loss: 16.9386 - val_accuracy: 0.9980
Epoch 8/100
1/1 [==============================] - 14s 14s/step - loss: 40.2044 - accuracy: 0.9967 - val_loss: 15.5189 - val_accuracy: 0.9980
Epoch 9/100
1/1 [=

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

In [ ]:



# Set a larger figure size
plt.figure(figsize=(10, 6))

# Plot the training and validation loss
plt.plot(history.history['loss'], color='blue', linestyle='-', linewidth=2, label='Training Loss')
plt.plot(history.history['val_loss'], color='orange', linestyle='--', linewidth=2, label='Validation Loss')

# Add title and labels
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

# Add a grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend to differentiate between training and validation loss
plt.legend(loc='best', fontsize=12)

# Set limits for better visualization
plt.xlim(0, len(history.history['loss']) - 1)  # From epoch 0 to the last epoch
plt.ylim(min(history.history['loss']) * 0.95, max(history.history['val_loss']) * 1.05)

# Show the plot
plt.tight_layout()
plt.show()
